# Evaluate and fit a ML model on the EC flux tower data 

Need to better consider categorical Landcover data. Currrently just passing it as integers in a single column. I can use one-hot-encoding but this may be a poor way of handling categorical variables in this case:
* https://notebook.community/roaminsight/roamresearch/BlogPosts/Categorical_variables_in_tree_models/categorical_variables_post
* https://notebook.community/marcotcr/lime/doc/notebooks/Tutorial_H2O_continuous_and_cat

* I should consider using `h2o` which handles categorical data much better https://www.kaggle.com/code/nanomathias/h2o-distributed-random-forest-starter/script
* R's randomforest algos do a better job too

* `LightGBM` handles categorical variables well: https://lightgbm.readthedocs.io/en/latest/Features.html#optimal-split-for-categorical-features


## Load modules

In [2]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import seaborn as sb
from joblib import dump
import multiprocessing
from pprint import pprint
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ShuffleSplit, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import warnings
warnings.filterwarnings("ignore")

## Analysis Parameters

In [3]:
model_name = 'AUS'
model_var = 'GPP'

In [4]:
ncpus=multiprocessing.cpu_count()
print('ncpus = '+str(ncpus))

ncpus = 16


## Prepare Data

In [5]:
base = '/g/data/os22/chad_tmp/NEE_modelling/results/training_data/'
sites = os.listdir('/g/data/os22/chad_tmp/NEE_modelling/results/training_data/')

td = []
for site in sites:
    if '.csv' in site:
        xx = pd.read_csv(base+site, index_col='time', parse_dates=True)
        td.append(xx)

## Modelling

In [6]:
xx = []
yy = []
for t in td:
    #t = t.drop('soil_moisture_RS', axis=1) # get rid of soil moisture 
    t = t.dropna()  # remove NaNS
    df = t.drop(['NEE_SOLO_EC','GPP_SOLO_EC','ER_SOLO_EC'], axis=1) # seperate carbon fluxes
    df = df.filter(regex='RS') # only use remote sensing variables   
    
    # Write out predictior variables to text file
    textfile = open("/g/data/os22/chad_tmp/NEE_modelling/results/variables.txt", "w")
    for element in df.columns:
        textfile.write(element + ",")
    textfile.close()
    
    df_var=t[model_var+'_SOLO_EC'] # seperate out the variable we're modelling

    x = df.reset_index(drop=True)#.to_numpy()
    y = df_var.reset_index(drop=True)#.to_numpy()
    xx.append(x)
    yy.append(y)

x = pd.concat(xx)
y = pd.concat(yy)

## Reclassify LC numbers to string


In [15]:
x['PFT_RS'] = x['PFT_RS'].astype(int)

In [ ]:
# x['PFT_RS'] = np.where(x['PFT_RS']==10.0, 'Trees', df_xx['PFT_RS'])
# x['PFT_RS'] = np.where(x['PFT_RS']=='20.0', 'Shrubs', df_xx['PFT_RS'])
# x['PFT_RS'] = np.where(x['PFT_RS']=='30.0', 'Grass', df_xx['PFT_RS'])
# x['PFT_RS'] = np.where(x['PFT_RS']=='40.0', 'Crops', df_xx['PFT_RS'])

# x = df_xx.to_numpy()
# y = df_yy.to_numpy()

## Test model robustness with nested K-fold cross validation

* Need to understand parameters of LightGBM better to define a sensible `param_grid`, and prevent overfitting
* If you set boosting as RF then the lightgbm algorithm behaves as random forest. According to the documentation, to use RF you must use bagging_fraction and feature_fraction smaller than 1

In [11]:
inner_cv_splits = 5

outer_cv_splits = 5

test_size = 0.20

In [19]:
# Create the parameter grid based on the results of random search 
# param_grid = {
#     'num_leaves': [7, 14, 21, 28, 31, 50, 70],
#     'boosting_type ': ['gbdt','dart'],
#     'max_depth': [-1, 3, 5, 10, 20],
#     'n_estimators': [100, 200, 300],
# }

# Create the parameter grid based on the results of random search 
param_grid = {
    # 'num_leaves': [7, 14, 21, 28, 31, 50, 70],
    # 'boosting_type ': ['rf'],
    'max_depth': [-1, 3, 5, 10, 20],
    'n_estimators': [100, 200, 300],
}

In [ ]:
outer_cv = KFold(n_splits=outer_cv_splits, shuffle=True,
                        random_state=0)

# lists to store results of CV testing
acc = []
rmse=[]
r2=[]
i = 1
for train_index, test_index in outer_cv.split(x, y):
    print(f"Working on {i}/5 outer cv split", end='\r')
    model = LGBMRegressor(random_state=1, n_jobs=ncpus)
    
    # index training, testing, and coordinate data
    X_tr, X_tt = x.iloc[train_index, :], x.iloc[test_index, :]
    y_tr, y_tt = y.iloc[train_index], y.iloc[test_index]
    
    # inner split on data within outer split
    inner_cv = KFold(n_splits=inner_cv_splits,
                     shuffle=True,
                     random_state=0)
    
    clf = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        scoring='r2',
        n_jobs=ncpus,
        refit=True,
        cv=inner_cv.split(X_tr, y_tr),
    )

    clf.fit(X_tr, y_tr,  categorical_feature=['PFT_RS'])
    # predict using the best model
    best_model = clf.best_estimator_
    pred = best_model.predict(X_tt)

    # evaluate model w/ multiple metrics
    # r2
    r2_ = r2_score(y_tt, pred)
    r2.append(r2_)
    # Overall accuracy
    ac = mean_absolute_error(y_tt, pred)
    acc.append(ac)
    # F1 scores
    rmse_ = np.sqrt(mean_squared_error(y_tt, pred))
    rmse.append(rmse_)
    i += 1

In [ ]:
print("Mean MAE % accuracy: "+ str(round(np.mean(acc), 2)))
print("Std dev of MAE % accuracy: "+ str(round(np.std(acc), 2)))
print('\n')
print("Mean RMSE: "+ str(round(np.mean(rmse), 2)))
print("Std dev RMSE: "+ str(round(np.std(rmse), 2)))
print('\n')
print("Mean r2: "+ str(round(np.mean(r2), 2)))
print("Std dev r2: "+ str(round(np.std(r2), 2)))

## Optimize model

In [25]:
#generate n_splits of train-test_split
rs = ShuffleSplit(n_splits=outer_cv_splits, test_size=test_size, random_state=1)

#instatiate a gridsearchCV
clf = GridSearchCV(LGBMRegressor(boosting_type='rf'),
                   param_grid,
                   scoring='r2',
                   verbose=1,
                   cv=rs.split(x, y),
                   n_jobs=ncpus)

clf.fit(x, y, categorical_feature=['PFT_RS'])

Fitting 5 folds for each of 15 candidates, totalling 75 fits


/home/547/cb3058/.local/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/547/cb3058/.local/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/547/cb3058/.local/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /__w/1/s/python-package/compile/src/boosting/rf.hpp, line 35 .

/home/547/cb3058/.local/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
[LightGBM] [Fatal] Check failed: config->bagging_freq > 0 && config->bagg

LightGBMError: Check failed: config->bagging_freq > 0 && config->bagging_fraction < 1.0f && config->bagging_fraction > 0.0f at /__w/1/s/python-package/compile/src/boosting/rf.hpp, line 35 .



[LightGBM] [Warning] Unknown parameter: dart
[LightGBM] [Warning] boosting_type is set=gbdt, boosting_type= will be ignored. Current value: boosting_type=gbdt
[LightGBM] [Warning] Unknown parameter: dart
[LightGBM] [Warning] boosting_type is set=gbdt, boosting_type= will be ignored. Current value: boosting_type=gbdt
[LightGBM] [Warning] Unknown parameter: dart
[LightGBM] [Warning] boosting_type is set=gbdt, boosting_type= will be ignored. Current value: boosting_type=gbdt
[LightGBM] [Warning] Unknown parameter: dart
[LightGBM] [Warning] boosting_type is set=gbdt, boosting_type= will be ignored. Current value: boosting_type=gbdt
[LightGBM] [Warning] Unknown parameter: dart
[LightGBM] [Warning] boosting_type is set=gbdt, boosting_type= will be ignored. Current value: boosting_type=gbdt
[LightGBM] [Warning] Unknown parameter: dart
[LightGBM] [Warning] boosting_type is set=gbdt, boosting_type= will be ignored. Current value: boosting_type=gbdt
[LightGBM] [Warning] Unknown parameter: dart


In [21]:
print("The most accurate combination of tested parameters is: ")
pprint(clf.best_params_)
print('\n')
print("The r2 score using these parameters is: ")
print(round(clf.best_score_, 2))

The most accurate combination of tested parameters is: 
{'max_depth': -1, 'n_estimators': 100}


The r2 score using these parameters is: 
nan


## Fit on all data using best params

In [ ]:
model = LGBMRegressor(**clf.best_params_)
model.fit(x, y, categorical_feature=['PFT_RS'])

In [ ]:
order = np.argsort(model.feature_importances_)

plt.figure(figsize=(15,4))
plt.bar(x=np.array(x.columns.values)[order],
        height=model.feature_importances_[order])
plt.gca().set_ylabel('Importance', labelpad=10)
plt.gca().set_xlabel('Feature', labelpad=10)
plt.tight_layout()

## Save the model

In [ ]:
dump(model, '/g/data/os22/chad_tmp/NEE_modelling/results/models/'+model_name+'_'+model_var+'_LGBM_model.joblib')

## Predict on all the data

In [ ]:
y_pred = model.predict(x)

rmse = float(format(np.sqrt(mean_squared_error(y, y_pred)), '.3f'))
print("RMSE:", rmse);

In [ ]:
sb.scatterplot(x=y,y=y_pred,color="#338844", edgecolor="white", s=50, lw=1, alpha=0.5)
sb.regplot(x=y, y=y_pred, scatter=False, color='m')
sb.regplot(x=y, y=y, color='black', scatter=False, line_kws={'linestyle':'dashed'});
plt.xlabel('True')
plt.ylabel('Prediction');